<h3> Prediction file structure: </h3>

```python
import joblib
results = joblib.load(<video.output_dir>/results/<video_name>.pkl)
results = {
  # A dictionary for each frame.
  'vid_frame0.jpg': {
    '2d_joints':       List[np.array(90,)],   # 45x 2D joints for each detection
    '3d_joints':       List[np.array(45,3)],  # 45x 3D joints for each detection
    'annotations':     List[Any],             # custom annotations for each detection
    'appe':            List[np.array(4096,)], # appearance features for each detection
    'bbox':            List[[x0 y0 w h]],     # 2D bounding box (top-left corner and dimensions) for each track (detections + ghosts)
    'camera':          List[[tx ty tz]],      # camera translation (wrt image) for each detection
    'camera_bbox':     List[[tx ty tz]],      # camera translation (wrt bbox) for each detection
    'center':          List[[cx cy]],         # 2D center of bbox for each detection
    'class_name':      List[int],             # class ID for each detection (0 for humans)
    'conf':            List[float],           # confidence score for each detection
    'frame_path':      'vid_frame0.jpg',      # Frame identifier
    'loca':            List[np.array(99,)],   # location features for each detection
    'mask':            List[mask],            # RLE-compressed mask for each detection
    'pose':            List[np.array(229,)],  # pose feature (concatenated SMPL params) for each detection
    'scale':           List[float],           # max(width, height) for each detection
    'shot':            int,                   # Shot number
    'size':            List[[imgw imgh]],     # Image dimensions for each detection
    'smpl':            List[Dict_SMPL],       # SMPL parameters for each detection: betas (10), body_pose (23x3x3), global_orient (3x3)
    'tid':             List[int],             # Track ID for each detection
    'time':            int,                   # Frame number
    'tracked_bbox':    List[[x0 y0 w h]],     # 2D bounding box (top-left corner and dimensions) for each detection
    'tracked_ids':     List[int],             # Track ID for each detection
    'tracked_time':    List[int],             # for each detection, time since it was last seen
  },
  'vid_frame1.jpg': {
    ...
  },
  ...
}
```

In [3]:
import numpy as np
import joblib
import matplotlib.pyplot as plt
import os

def proc(path, focus_idx=0):
    results = joblib.load(path)
    print("[#INFO] From: ", path)
    print("[#INFO] #N frames: ", len(results))
    tracking_presence = {}
    for fid, data in results.items():
        for pid in data['tracked_ids']:
            if pid not in tracking_presence:
                tracking_presence[pid] = 1
            else:
                tracking_presence[pid] += 1
    longest = max(tracking_presence.values())
    longest_id = [k for k, v in tracking_presence.items() if v == longest][focus_idx]
    print("[#INFO] #N unique ids: ", len(tracking_presence))
    print("[#INFO] Longest tracking id: ", longest_id)
    
    all_j3d = []
    betas = []
    body_pose = []
    global_orient = []
    for fid, data in results.items():
        if len(data['tid']) == 0:
            continue
        pid = data['tid'].index(longest_id)
        # print(data['conf'])
        # print(data['class_name'])
        assert data['class_name'][pid] == 0   # person = 0
        assert data['conf'][pid] > 0.85
        
        j3d = data['3d_joints'][pid]    # [45, 3]
        all_j3d.append(j3d)
        
        smpl = data['smpl'][pid]    # [45, 3]
        betas.append(smpl['betas'])
        body_pose.append(smpl['body_pose'])
        global_orient.append(smpl['global_orient'])
        
        # print(data['pose'][pid].shape)
        # for k, v in data['smpl'][pid].items():
        #     print(k, v.shape)
        
        # assert False
        
    all_j3d = np.stack(all_j3d)  # [T, 45, 3]
    betas = np.stack(betas)  # [T, 10]
    body_pose = np.stack(body_pose)  # [T, 23, 3, 3]
    global_orient = np.stack(global_orient)  # [T, 1, 3, 3]
    print("[#INFO] all_j3d shape: ", all_j3d.shape)
    print("[#INFO] betas shape: ", betas.shape)
    print("[#INFO] body_pose shape: ", body_pose.shape)
    print("[#INFO] global_orient shape: ", global_orient.shape)
    
    out = {'3d_joints': all_j3d, 'betas': betas, 'body_pose': body_pose, 'global_orient': global_orient}
    # import plotly.graph_objects as go
    # x = all_j3d
    # fig = go.Figure(data=[go.Scatter3d(x=x[0, :, 0], y=x[0, :, 1], z=x[0, :, 2], mode='markers')])
    # fig.update_layout(scene=dict(
    #     xaxis=dict(nticks=4, range=[-1, 1]),
    #     yaxis=dict(nticks=4, range=[-1, 1]),
    #     zaxis=dict(nticks=4, range=[-1, 1])
    # ))
    # fig.write_html('./test.html')
    
    print(all_j3d.shape)
    # Write to pkl
    # out_path = os.path.join(os.path.dirname(path), 'out.pkl')
    out_path = os.path.join(f"./{os.path.basename(path).split('.')[0]}.pkl".format(longest_id))
    joblib.dump(out, out_path)
    
        
        
        
        
# path = '/home/mint/Dev/CR7/pose_estimators/3D/4D-Humans/outputs/results/demo_cr7_1.pkl'
path = '/home/mint/Dev/CR7/pose_estimators/3D/4D-Humans/outputs/results/demo_weightlifting.pkl'
proc(path)

[#INFO] From:  /home/mint/Dev/CR7/pose_estimators/3D/4D-Humans/outputs/results/demo_weightlifting.pkl
[#INFO] #N frames:  287
[#INFO] #N unique ids:  3
[#INFO] Longest tracking id:  1
[#INFO] all_j3d shape:  (287, 45, 3)
[#INFO] betas shape:  (287, 10)
[#INFO] body_pose shape:  (287, 23, 3, 3)
[#INFO] global_orient shape:  (287, 1, 3, 3)
(287, 45, 3)
